## Import Statements

In [11]:
import gensim.downloader as api
import numpy as np
import re
import csv
import pandas as pd
import pprint
import string
import nltk
import sys
import spacy

from IPython.display import HTML
from nltk.corpus import wordnet 
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize

pathToDatasets = '../datasets/'
pathToDataScripts = '../datasets/scripts/'
filePath = '../datasets/GoogleNews-vectors-negative300.bin'

sys.path.insert(0, pathToDataScripts)
from cleanDataset import tokenize_words 


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

## Downloading binaries and models


In [12]:
print("Should I reload the model?")
tstString = input()
if("no" in tstString.lower()):
    print(" didnt reload model! ")
else:
    print("loading the model!");
    word_vectors = api.load("glove-wiki-gigaword-100")
    nltk.download('vader_lexicon')
    nltk.download('punkt')
    nltk.download('averaged_perceptron_tagger')
    nltk.download('wordnet')

Should I reload the model?
no
 didnt reload model! 


## Global Variables and Global Objects

In [3]:
senty = SentimentIntensityAnalyzer()
vocabulary = word_vectors.vocab;

nlp = spacy.load('en')


NUMBER_OF_ALTERNATIVES = 5
TWEET_START = 104
NUM_OF_TWEETS = 5


# VERBOSE_PRINTING = True
VERBOSE_PRINTING = False

USE_SPACY = False
# USE_SPACY = True

COLOR_PRINTING = True
#COLOR_PRINTING = False



punctuation = r"\"#$%&'+-/;<=>?@[\]*.^_`{|}~"

## Class for Sentences


In [4]:
class SentenceWithHTML():
    
    def __init__(self,sentence,html):
        self.sentence = sentence;
        self.html = html;
    
    def getHTML(self):
        return self.html;
    
    def getSentence(self):
        return self.sentence
    
    def setHTML(self, html):
        self.html = html;
    
    def setSentence(self, sentence):
        self.sentence = sentence


class Sentence:
    
    
    def __init__(self, sentence, sentiment):
        self.ogSentence = sentence;
        self.ogSentiment = sentiment;
        self.indexToSetOfWords = {}
        self.alternateSentences = [];
        self.finalShiftSentences = [];

    def addAlternativesByIndex(self, index, listOfAlternatives):
        '''
            Adds the list of possible alternative words that 
            can be used per word based on the index of the word in the tokenized 
            sentence. (from cleanAndTokenizeText())
        '''
        if(self.indexToSetOfWords.get(index)):
            self.indexToSetOfWords[index] = self.indexToSetOfWords.union(set(listOfAlternatives))
        else:
            self.indexToSetOfWords[index] = set(listOfAlternatives)
        
    def addAlternativeStrings(self, strings):
        if(isinstance(strings,str)):
            self.alternateStrings = list(set(self.alternateStrings.append(strings)))
            self.alternateSentences.append(strings)
        else:
            self.alternateSentences.extend(strings)
    

    
    def addFinalSentences(self, sentences):
        if(isinstance(sentences, str)):
            self.finalShiftSentences.append(sentences)
        else:
            self.finalShiftSentences.extend(sentences)

    def resetFinalSentences(self):
        self.finalShiftSentences = [];
        
            

## Utility Code

In [5]:

def printStrings(sentenceObj):
    newStrings = sentenceObj.finalShiftSentences;
    mainSentiment = sentenceObj.ogSentiment;
    for ind, tSentence in enumerate(newStrings):
            alteredTweet = tSentence.getSentence()
            htmlText = tSentence.getHTML()
            sentimentOfNewString = senty.polarity_scores(alteredTweet)['compound']
            if(sentimentOfNewString == mainSentiment or sentimentOfNewString == 0):
                displayText("{0}: {1}".format(htmlText,sentimentOfNewString), 'DarkGray');
            elif(sentimentOfNewString > mainSentiment):
                displayText("{0}: {1}".format(htmlText,sentimentOfNewString),'green')
            else:
                displayText("{0}: {1}".format(htmlText,sentimentOfNewString),'red')

In [177]:
def cstr(s, color='black', italics=False):
    if(COLOR_PRINTING):
        if(italics):
            return cstr("<i>{0}</i>".format(s), color);
        return "<text style=color:{}>{}</text>".format(color, s)
    else:
        return "{}".format(s)

def displayText(text, color='black'):
    if(COLOR_PRINTING):
        display(HTML(cstr(text, color)));
        return
    print("{}".format(text));
    
    
def cleanAndTokenizeText(text):
    text = text.lower();
    newString = ""
    for char in text:
        if char not in punctuation:
            newString += char
    text = word_tokenize(newString)
    return text;

def getPOSTags(tweet):
    if(USE_SPACY == False):
        tags = nltk.pos_tag(tweet)
        return tags;    
    tweet = ' '.join(tweet)
    doc = nlp(tweet)
    tags = [(token.text, token.pos_) for token in doc] # since the format expected is [text,tag]
    return tags;
    

def getAntonymsOfWords(word):
    if(word not in vocabulary):
        return []
    setOfAntonyms = set()
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            anton = l.antonyms()
            if(anton!=[]):
                setOfAntonyms.add(anton[0].name())
    if(len(setOfAntonyms) == 0):
        if(VERBOSE_PRINTING): print("No antonyms found for word {0}".format(word))
    return list(setOfAntonyms)

def listReplacements(word):
    if(word not in vocabulary):
        print(" --- {0} not in vocabulary ---".format(word))
        return []
    possibleReplacements = [word[0] for word in word_vectors.most_similar(word,topn=NUMBER_OF_ALTERNATIVES)]
    if(possibleReplacements == []):
        print(" --- No replacements for word {0} ---".format(word))
    antonyms = getAntonymsOfWords(word)
    if(antonyms != []):
        possibleReplacements.extend(antonyms)
        if(VERBOSE_PRINTING): print("Some antonyms for word {0} are {1}".format(word, antonyms[:3]))
        return possibleReplacements
    return possibleReplacements
    
def posApprovedReplacements(alternativeWords, userTokens, indexOfToken):
    if(alternativeWords == []):
        return []
    tempTokens = userTokens[:]
    POSTokens = getPOSTags(tempTokens)
    validWords = []
    
    mainTag = POSTokens[indexOfToken][1]
    mainWord = userTokens[indexOfToken]
    
    for ind,word in enumerate(alternativeWords):
        tempTokens[indexOfToken] = word
        posTags = getPOSTags(tempTokens)
        newTag = (posTags[indexOfToken])[1]
        
        if(str(newTag) == str(mainTag)):
            if(VERBOSE_PRINTING): print("Word {0}[{1}] replaced with {2}[{3}]".format(mainWord, mainTag, word,newTag))
            validWords.append(word)
    if(validWords == [] and VERBOSE_PRINTING):
        print("No POS words found for word {} with tag {}".format(mainWord, mainTag));
    return validWords
        
    
    
def getAlternativeSentences(sentenceObj):
    mainSentence = sentenceObj.ogSentence;
    mainSentiment = sentenceObj.ogSentiment;

    sentenceTokens = cleanAndTokenizeText(mainSentence)

    for ind, word in enumerate(sentenceTokens):
        alternativeSentenceWithHTML = []

        score = senty.polarity_scores(word)['compound']
        copyOfTokens = sentenceTokens[:]
        replacements = []
        if(score != 0.0):
            tempReplacements = listReplacements(word) # get embedding based relations
            if(tempReplacements == []):
                print("No replacements found at all for word {0}".format(word))
                continue
            replacements = posApprovedReplacements(tempReplacements[:], copyOfTokens[:], ind)
            if(replacements == []):
                print(" -- No POS approved words! -- for word {0}\n some non-POS:{1}".format(word, tempReplacements[:4]))
                continue
            sentenceObj.addAlternativesByIndex(ind, replacements)

            ## Generate new sentences by switching that word
            for newWord in replacements:
                htmlFriendlyTokens = copyOfTokens[:]
                copyOfTokens[ind] = newWord
                htmlFriendlyTokens[ind] = cstr("[{0}]".format(newWord), "blue", italics=True);
                newString = ' '.join(copyOfTokens)
                tSentence = SentenceWithHTML(newString, ' '.join(htmlFriendlyTokens))
                alternativeSentenceWithHTML.append(tSentence)
        sentenceObj.addAlternativeStrings(alternativeSentenceWithHTML)
    return sentenceObj

def shiftSentiment(sentenceObj, positive=True):
    
    actualTweet = sentenceObj.ogSentence;
    mainSentiment = sentenceObj.ogSentiment;
    alternateTweets = sentenceObj.alternateSentences;
    
    
    happiestTweet = ""
    saddestTweet = ""
    happiestScore = -sys.maxsize - 1
    saddestScore = sys.maxsize
    correctTweets = []
    
    for tSentence in alternateTweets:
        tweet = tSentence.getSentence();
        newSenty = senty.polarity_scores(tweet)['compound']
        
        if(newSenty < saddestScore):
            saddestTweet = tSentence
            saddestScore = newSenty
        if(newSenty > happiestScore):
            happiestTweet = tSentence
            happiestScore = newSenty
            
        if(newSenty == mainSentiment):
            continue
        if(positive == True):
            if(newSenty > mainSentiment):
                correctTweets.append(tSentence)
                continue
            
            elif(newSenty < mainSentiment):
                continue
                # Grab happiest tweet and if it's not "", then generate more happy tweets from it
        if(positive == False):
            if(newSenty < mainSentiment):
                correctTweets.append(tSentence)
                continue
            
            elif(newSenty > mainSentiment):
                continue
                # grab happiest tweet, and if it's not "", then generate more happy tweets from it
                
    if(correctTweets == []):
        print("\n\nNo tweets found when trying to do Positive={}\n\n".format(positive))
    sentenceObj.resetFinalSentences();
    sentenceObj.addFinalSentences(correctTweets);
    return sentenceObj;


def helper_Combine(mainList, myList):
    newList = []
    for val in myList:
        for mainVal in mainList:
            if(VERBOSE_PRINTING): print("Combining {0} with {1}".format(' '.join(val), ' '.join(mainVal)));
            newList.append(val + mainVal);
    return newList;


def combineSentenceChunks(wholeSentence, dictOfChunks):
    reversedKeys = sorted(dictOfChunks.keys(), reverse=True)
    completeSentences = [];
    mainArr = dictOfChunks[reversedKeys[0]]
    for ind in range(1, len(reversedKeys)):
        key = reversedKeys[ind];
        mainArr = helper_Combine(mainArr, dictOfChunks[key]);
        
    return mainArr;
    
        
def generateSentenceChunks(wholeSentence, keyToChange, nextKey, listOfMyAlternatives):
    newList = list(listOfMyAlternatives)
    newList.append(wholeSentence[keyToChange]);
    generatedSentences = []
    generatedSentenceObjects = []
    for myAlt in newList:
        newSentence = wholeSentence[:]
        newSentence[keyToChange] = myAlt
        htmlSentence = newSentence[:]
        htmlSentence[keyToChange] = cstr("[{0}]".format(newWord), "blue", italics=True);
        if(VERBOSE_PRINTING): print("Generated : {}".format(newSentence[keyToChange:nextKey]))
        generatedSentences.append(newSentence[keyToChange:nextKey]);
        generatedSentenceObjects.append(SentenceWithHTML(' '.join(htmlSentence),' '.join(newSentence[keyToChange:nextKey])))
        
        
    return generatedSentences, generatedSentenceObjects
    
    
def printAllPossibleStrings(sentenceObj):
    indexToWordDict = sentenceObj.indexToSetOfWords;
    originalSentence = sentenceObj.ogSentence;
    tokenizedSentence = cleanAndTokenizeText(originalSentence)
    reversedKeys = sorted(indexToWordDict.keys(), reverse=True)
    dictAlternatives  = {}

    keys = sorted(indexToWordDict.keys())
    sentenceChunks = {}
    htmlChunks = {}
    print("Keys : {0}".format(keys))
    
    for ind in range(0,len(keys)):
        key = keys[ind]
        nextKey = keys[ind+1] if ind+1 < len(keys) else len(tokenizedSentence)
        sentenceChunks[key], htmlChunks[key] = generateSentenceChunks(tokenizedSentence, key, nextKey, indexToWordDict[key])
    
    if(keys[0] != 0):
        newList = []
        for thislist in sentenceChunks[keys[0]]:
            newList.append(tokenizedSentence[0:keys[0]] + thislist)
        sentenceChunks[keys[0]] = newList;
        
    print("Sentence Chunks generated!")
        
        
    finalOptions = combineSentenceChunks(tokenizedSentence, sentenceChunks)
    
    
    for val in finalOptions:
        sentence = ' '.join(val)
        score = senty.polarity_scores(sentence)['compound']
        if(score == 0.0):
            continue
        print("{} : {}".format(sentence, score))
        
#     for val in totalList:
#         print(' '.join(val))


In [178]:
specificString = ""
def specificString(textString=""):
    if(textString == "" or textString == None):
        return
    mainSentiment = senty.polarity_scores(textString)['compound']
    if(mainSentiment == 0):
        print("{} \n No sentiment found in sentence".format(textString));
        return;
    print("\n {0}:{1}\n".format(textString,mainSentiment))   
    sentenceObj = Sentence(textString, mainSentiment)
    sentenceObj = getAlternativeSentences(sentenceObj)
    alternateTweets = (sentenceObj.alternateSentences)[:]
    if(alternateTweets == [] or alternateTweets == None):
        print(" -- No new Strings generated ---\n\n")
        return
    printStrings(sentenceObj)
    printAllPossibleStrings(sentenceObj)

# specificString("I really hate hot chocolate, but i enjoy hot coffee, and I'm serious")
specificString("Keep calm and carry on debugging till it works");
# specificString("would rather the first party send bad messages than the 3rd party send mixed ones  sophmore year all over again")


 Keep calm and carry on debugging till it works:0.3182

Keys : [1]
Sentence Chunks generated!
keep tense and carry on debugging till it works : -0.34
keep relaxed and carry on debugging till it works : 0.4939
keep stimulate and carry on debugging till it works : 0.2263
keep agitate and carry on debugging till it works : -0.4019
keep calm and carry on debugging till it works : 0.3182


In [179]:
def runThroughTweets():
    
    tweets_df = pd.read_csv( pathToDatasets + 'cleanedTweets.csv' , nrows=NUM_OF_TWEETS, skiprows=TWEET_START)

    tweets = tweets_df.values

    listOfObjects = []
    for counter,tweet in enumerate(tweets):
        tweet = tweet[0]
        mainSentiment = senty.polarity_scores(tweet)['compound']
        if(mainSentiment == 0):
            continue
        print("\n {0}:{1}\n".format(tweet,mainSentiment))   
        sentenceObj = Sentence(tweet, mainSentiment)
        sentenceObj = getAlternativeSentences(sentenceObj)
        alternateTweets = (sentenceObj.alternateSentences)[:]
        if(alternateTweets == [] or alternateTweets == None):
            print(" -- No new Strings generated ---\n\n")
            continue
        sentenceObj = shiftSentiment(sentenceObj, True);
        printStrings(sentenceObj)
        sentenceObj = shiftSentiment(sentenceObj, False);
        printStrings(sentenceObj)
        printAllPossibleStrings(sentenceObj)
    
runThroughTweets()


  remember my bum leg strikes back this time its serious :-0.4215



Keys : [4, 9]
Sentence Chunks generated!
remember my bum leg airstrikes back this time its severe : -0.3818
remember my bum leg airstrikes back this time its serious : -0.0772
remember my bum leg attacks back this time its possible : -0.4404
remember my bum leg attacks back this time its frivolous : -0.4404
remember my bum leg attacks back this time its severe : -0.6705
remember my bum leg attacks back this time its serious : -0.4939
remember my bum leg raids back this time its severe : -0.3818
remember my bum leg raids back this time its serious : -0.0772
remember my bum leg strikes back this time its possible : -0.3612
remember my bum leg strikes back this time its frivolous : -0.3612
remember my bum leg strikes back this time its severe : -0.6249
remember my bum leg strikes back this time its serious : -0.4215

  cool i will. their are all kinds of complaints about this laptop online about overheating but no recalls :-0.4588



Keys : [0, 8, 16]
Sentence Chunks generated!
chill i will their are all kinds of cases about this laptop online about overheating but no recalls : -0.4215
chill i will their are all kinds of allegations about this laptop online about overheating but no recalls : -0.4215
chill i will their are all kinds of inquiries about this laptop online about overheating but no recalls : -0.4215
chill i will their are all kinds of accusations about this laptop online about overheating but all recalls : -0.1655
chill i will their are all kinds of accusations about this laptop online about overheating but any recalls : -0.1655
chill i will their are all kinds of accusations about this laptop online about overheating but no recalls : -0.5346
chill i will their are all kinds of complaints about this laptop online about overheating but all recalls : -0.2144
chill i will their are all kinds of complaints about this laptop online about overheating but any recalls : -0.2144
chill i will their are all kinds 

Keys : [3, 14]
Sentence Chunks generated!
emily will be happy when mommy is done training at her new job she throws her : 0.5719
emily will be happy when mommy is done training at her new job she chances her : 0.6705
emily will be happy when mommy is done training at her new job she kicks her : 0.5719
emily will be happy when mommy is done training at her new job she misses her : 0.4215
emily will be sad when mommy is done training at her new job she throws her : -0.4767
emily will be sad when mommy is done training at her new job she chances her : -0.3182
emily will be sad when mommy is done training at her new job she kicks her : -0.4767
emily will be sad when mommy is done training at her new job she misses her : -0.6124
emily will be thankful when mommy is done training at her new job she throws her : 0.5719
emily will be thankful when mommy is done training at her new job she chances her : 0.6705
emily will be thankful when mommy is done training at her new job she kicks her : 0.5

Keys : [4, 6, 11]
Sentence Chunks generated!
would rather the first leader send unregretful messages than the 3rd party send mixed ones sophmore year all over again : 0.4019
would rather the first leader send good messages than the 3rd leader send mixed ones sophmore year all over again : 0.4404
would rather the first leader send good messages than the 3rd coalition send mixed ones sophmore year all over again : 0.4404
would rather the first leader send good messages than the 3rd opposition send mixed ones sophmore year all over again : 0.4404
would rather the first leader send good messages than the 3rd party send mixed ones sophmore year all over again : 0.6808
would rather the first leader send bad messages than the 3rd leader send mixed ones sophmore year all over again : -0.5423
would rather the first leader send bad messages than the 3rd coalition send mixed ones sophmore year all over again : -0.5423
would rather the first leader send bad messages than the 3rd opposition send mi

## Testing Code


In [9]:
specificWord = "good"
def testOneWord(word=""):
    if(word==""):
        return
    print(word)

## Main Cell
